## Import libraries

In [50]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Preprocess Data

In [51]:
train_data = np.load('dataset/Glasses & Sunglasses - Train.npz')
validation_data = np.load('dataset/Glasses & Sunglasses - Validation.npz')
test_data = np.load('dataset/Glasses & Sunglasses - Test.npz')

In [52]:
images_train, labels_train = train_data['images'], train_data['labels']
images_val, labels_val = validation_data['images'], validation_data['labels']
images_test, labels_test = test_data['images'], test_data['labels']

In [53]:
labels_train

array([0, 1, 1, ..., 1, 1, 0])

In [54]:
images_train.shape

(4002, 120, 90, 3)

In [55]:
labels_train.shape

(4002,)

In [56]:
images_val.shape

(500, 120, 90, 3)

In [57]:
images_test.shape

(500, 120, 90, 3)

In [58]:
labels_val.shape

(500,)

In [59]:
labels_test.shape

(500,)

In [60]:
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Create the model

### Define some hyperparameters

In [61]:
BATCH_SIZE = 64
EPOCHS = 15

In [62]:
# hyperparameter tuning
HP_KERNEL_SIZE = hp.HParam('kernel size', hp.Discrete([3,5,7]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([32,64,96,128]))

METRIC = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 1 (Glasses&Sunglasses)/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE, HP_KERNEL_NUM],
        metrics = [hp.Metric(METRIC, display_name='accuracy')]
    )

In [63]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE], activation = 'relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], 3, activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2)
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy'])
    
    log_dir = "Logs\\Model 1 (Glasses&Sunglasses)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Glasses','Sunglasses'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    #callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True)
    tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1, profile_batch = 0)
    cm_callbacks = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    
    model.fit(images_train,
              labels_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callbacks, cm_callbacks, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1)
    
    _, accuracy = model.evaluate(images_val, labels_val)
    
    model.save(r'saved_models\Model 1 (Glasses&Sunglasses)\Run-{}'.format(session_num))
    
    return accuracy

In [64]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [65]:
session_num = 1
# all hparams combination
for kernel_size in HP_KERNEL_SIZE.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        hparams = {
            HP_KERNEL_SIZE : kernel_size,
            HP_KERNEL_NUM : kernel_num
        }
        run_name = f"run-{session_num}"
        print('--- Starting Trial:',run_name)
        print({h.name: hparams[h] for h in hparams})
        run("Logs/Model 1 (Glasses&Sunglasses)/hparam_tuning/" + run_name, hparams, session_num)
        
        session_num += 1

--- Starting Trial: run-1
{'kernel size': 3, 'kernel num': 32}
Epoch 1/15
63/63 [==============================] - 18s 289ms/step - loss: 0.4622 - accuracy: 0.7706 - val_loss: 0.2372 - val_accuracy: 0.9440
Epoch 2/15
63/63 [==============================] - 18s 287ms/step - loss: 0.1595 - accuracy: 0.9465 - val_loss: 0.2229 - val_accuracy: 0.9580
Epoch 3/15
63/63 [==============================] - 18s 283ms/step - loss: 0.1499 - accuracy: 0.9518 - val_loss: 0.2062 - val_accuracy: 0.9540
Epoch 4/15
63/63 [==============================] - 18s 279ms/step - loss: 0.1364 - accuracy: 0.9575 - val_loss: 0.2082 - val_accuracy: 0.9500
Epoch 5/15
63/63 [==============================] - 17s 277ms/step - loss: 0.1358 - accuracy: 0.9580 - val_loss: 0.2037 - val_accuracy: 0.9560
Epoch 6/15
63/63 [==============================] - 17s 277ms/step - loss: 0.1188 - accuracy: 0.9650 - val_loss: 0.2062 - val_accuracy: 0.9580
Epoch 7/15
16/16 [==============================] - 1s 38ms/step - loss: 0.2037

63/63 [==============================] - 55s 878ms/step - loss: 0.3875 - accuracy: 0.8263 - val_loss: 0.2229 - val_accuracy: 0.9360
Epoch 2/15
63/63 [==============================] - 56s 882ms/step - loss: 0.1715 - accuracy: 0.9445 - val_loss: 0.2259 - val_accuracy: 0.9360
Epoch 3/15
63/63 [==============================] - 56s 881ms/step - loss: 0.1660 - accuracy: 0.9468 - val_loss: 0.2096 - val_accuracy: 0.9400
Epoch 4/15
63/63 [==============================] - 55s 879ms/step - loss: 0.1488 - accuracy: 0.9548 - val_loss: 0.2165 - val_accuracy: 0.9600
Epoch 5/15
63/63 [==============================] - 55s 877ms/step - loss: 0.1572 - accuracy: 0.9525 - val_loss: 0.2029 - val_accuracy: 0.9440
Epoch 6/15
63/63 [==============================] - 56s 884ms/step - loss: 0.1657 - accuracy: 0.9493 - val_loss: 0.2691 - val_accuracy: 0.9300
Epoch 7/15
16/16 [==============================] - 2s 107ms/step - loss: 0.2029 - accuracy: 0.9440
INFO:tensorflow:Assets written to: saved_models\Model